In [31]:
import os
from dotenv import load_dotenv
load_dotenv()

from uuid import uuid4

import time
from docx import Document

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from pinecone import ServerlessSpec, Pinecone

In [3]:
# setup
index_name = "simple-assignment-index"

In [10]:
# Ingestion
loader = PyPDFLoader("./files/syllabus.pdf")
docs = loader.load()
docs

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 40
)

chunks = splitter.split_documents(docs)

In [11]:
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [12]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    time.sleep(10)
    

In [18]:
vector_store= PineconeVectorStore(index_name=index_name, embedding=embeddings)
uuids = [ str(uuid4()) for _ in range(len(chunks))]
vector_store.add_documents(documents=chunks, id=uuids)

['ebb5ddfb-06f7-4348-8219-b26df7e2fda9',
 '7623acdb-e572-4176-a1ad-17880b2f0ae2',
 '315a41b7-7f6b-48fe-a0cb-4b6e5082abc3',
 'd8f36aaf-e47b-4405-972b-2067b13ef3cb',
 'fdde4445-2f33-4d1d-905f-a2e7d09dee9c',
 '925bfe63-63e3-47e0-a770-54e39435596c',
 'f0d6d15e-3e67-4128-8996-cded28b23bfc',
 '3d0f6ed1-fd28-497b-8901-573c35be94f8',
 'a4960add-843b-4d72-b943-4fd0de3a9764',
 '32d8a11a-7ae0-494f-8560-1298d01158a1',
 '90e1cc82-ddb2-460d-92ba-d94372e76459',
 '8c707717-fa99-49db-8e5c-4f7c9b9215ae',
 'fdb61a25-2a09-445b-bba9-564e7f18412b',
 '2d0df1e7-dcda-4724-88c4-6bf7bbb9d107',
 '523c15d5-3c47-4004-b2e1-38ad2b442cc3',
 'db9f8b19-e25b-4cb8-958a-8d7d700ed7e3',
 '82e5672d-40b7-4447-8146-89a3287814eb',
 'e5730dd9-c0b3-4fb0-84da-e282ad936834',
 'ebf8f266-d055-4283-9b28-623c18e17562',
 '8f15130e-87f3-4a61-b316-509785a1d83f',
 '725a3354-6ac3-42ab-9a8c-05bbf11c0807',
 'ef3362a0-716a-4c8d-8fdf-cc960bd238c5',
 'a95bcbde-e8ee-4fa4-a83e-f41a19916825',
 'e3cdaa16-f0ed-4e28-8a10-4568a5c0c011',
 '28b013ed-d2da-

In [25]:
retriver = vector_store.as_retriever(search_type ="mmr",search_kwargs={"k":3,"fetch_keys":4})

In [34]:
# model
model = ChatGroq(model="qwen/qwen3-32b")

#prompt
template= """
Answer the question based ONLY on the context.
Context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)



In [35]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [36]:
chain = (
    {"context": retriver | format_docs, "question":RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [37]:
# 6. EXECUTION & SAVING
query = "What are the safety evaluations performed on Llama 2?"
print(f"Answering: {query}")

Answering: What are the safety evaluations performed on Llama 2?


In [ ]:
start = time.time()
response = chain.invoke(query)
end = time.time()

# Save to Word
doc = Document()
doc.add_heading('Assignment Report', 0)
doc.add_paragraph(f"Query: {query}")
doc.add_paragraph(f"Answer: {response}")
doc.add_paragraph(f"Time Taken: {end - start:.2f} seconds")
doc.save('Simple_Report.docx')